<a href="https://colab.research.google.com/github/Ankur3107/colab_notebooks/blob/master/Generic_Transformer_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.3MB/s 
     |████████████████████████████████| 890kB 19.7MB/s 
     |████████████████████████████████| 3.0MB 20.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6bda503b3bbdbf7626ff38a3e3235c3a20e948d97c40c78a681f5c87b90ed237
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import os, pandas as pd
from sklearn.model_selection import train_test_split
import logging
from transformers import *
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm.autonotebook import tqdm

# 1. Set Configuration

In [4]:
class Config:
  train_file = './data.csv'
  eval_file = './eval.csv'
  max_seq_len = 128
  batch_size = 32
  epochs = 5
  model_name = 'bert-base-uncased'
  learning_rate = 2e-5
  n_classes = 3
  device = 'cpu'
  


flags = Config

# 2. Build Dataset Pipeline

In [5]:
class TextLabelDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
  
    def __len__(self):
        return len(self.texts)
  
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )

        return {
          'texts': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size, is_prediction=False):

  if isinstance(df, str):
    df = pd.read_csv(df)
  else:
    pass

  if is_prediction:
    ds = TextLabelDataset(
        texts=df.text.to_numpy(),
        labels=np.array([-1]*len(df.text.values)),
        tokenizer=tokenizer,
        max_len=max_len
        )
  else:
    ds = TextLabelDataset(
        texts=df.text.to_numpy(),
        labels=df.labels.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
        )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
        )

# 3. Build Model 

In [6]:
class Classifier(nn.Module):

  def __init__(self, model_name, n_classes):
      super(Classifier, self).__init__()
      self.bert = AutoModel.from_pretrained(model_name)
      self.drop = nn.Dropout(p=0.3)
      self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
      _, pooled_output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      output = self.drop(pooled_output)
      return self.out(output)

In [26]:
class ClassificationModel:

  def __init__(self, flags):
    self.flags = flags
    self.tokenizer = BertTokenizer.from_pretrained(self.flags.model_name)
    self.model = Classifier(self.flags.model_name, self.flags.n_classes)
    self.model = self.model.to(self.flags.device)

  def train(self):

    train_data_loader = create_data_loader(self.flags.train_file, self.tokenizer, self.flags.max_seq_len, self.flags.batch_size)
    val_data_loader = create_data_loader(self.flags.eval_file, self.tokenizer, self.flags.max_seq_len, self.flags.batch_size)

    optimizer = AdamW(self.model.parameters(), lr=self.flags.learning_rate, correct_bias=False)
    total_steps = len(train_data_loader) * self.flags.epochs

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=0,
      num_training_steps=total_steps
    )

    loss_fn = nn.CrossEntropyLoss().to(self.flags.device)

    history = defaultdict(list)
    best_accuracy = 0

    if isinstance(self.flags.train_file, str):
      train_df = pd.read_csv(self.flags.train_file)

    if isinstance(self.flags.eval_file, str):
      eval_df = pd.read_csv(self.flags.eval_file)

    for epoch in range(self.flags.epochs):

      print(f'Epoch {epoch + 1}/{self.flags.epochs}')
      print('-' * 10)

      train_acc, train_loss = self.train_epoch(
        self.model,
        train_data_loader,    
        loss_fn, 
        optimizer, 
        self.flags.device, 
        scheduler, 
        len(train_df)
      )

      print(f'Train loss {train_loss} accuracy {train_acc}')

      val_acc, val_loss = self.eval_model(
        self.model,
        val_data_loader,
        loss_fn, 
        self.flags.device, 
        len(eval_df)
      )

      print(f'Val   loss {val_loss} accuracy {val_acc}')
      print()

      history['train_acc'].append(train_acc)
      history['train_loss'].append(train_loss)
      history['val_acc'].append(val_acc)
      history['val_loss'].append(val_loss)

      if val_acc > best_accuracy:
        torch.save(self.model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc


  def train_epoch(self, model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()

    losses = []
    correct_predictions = 0
    tk0 = tqdm(data_loader, total=len(data_loader), desc="Training")
    for bi, d in enumerate(tk0):
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

  def eval_model(self, model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
      tk0 = tqdm(data_loader, total=len(data_loader), desc="Evaluating")
      for bi, d in enumerate(tk0):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        _, preds = torch.max(outputs, dim=1)

        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)


    

### Download Data and Preparation

In [8]:
!wget https://raw.githubusercontent.com/SrinidhiRaghavan/AI-Sentiment-Analysis-on-IMDB-Dataset/master/imdb_tr.csv

--2020-08-26 15:57:14--  https://raw.githubusercontent.com/SrinidhiRaghavan/AI-Sentiment-Analysis-on-IMDB-Dataset/master/imdb_tr.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23677025 (23M) [text/plain]
Saving to: ‘imdb_tr.csv’

imdb_tr.csv         100%[===================>]  22.58M  39.9MB/s    in 0.6s    

2020-08-26 15:57:16 (39.9 MB/s) - ‘imdb_tr.csv’ saved [23677025/23677025]



In [9]:
data = pd.read_csv('imdb_tr.csv', encoding = "ISO-8859-1")

In [10]:
data.columns = ['row_Number', 'text', 'labels']

In [11]:
train_data = data.sample(1000)
test_data = data.sample(100)


In [12]:
train_data.to_csv('data.csv', index=False)
test_data.to_csv('eval.csv', index=False)

# Training

In [27]:
from collections import defaultdict
import numpy as np

In [28]:
class Config:
  train_file = './data.csv'
  eval_file = './eval.csv'
  max_seq_len = 128
  batch_size = 32
  epochs = 5
  model_name = 'bert-base-uncased'
  learning_rate = 2e-5
  n_classes = 2
  device = 'cuda'
  
flags = Config

In [29]:
classification = ClassificationModel(flags)

In [30]:
classification.train()

Epoch 1/5
----------



Train loss 0.6540139000862837 accuracy 0.622



Val   loss 0.4141501262784004 accuracy 0.78

Epoch 2/5
----------



Train loss 0.3276493112789467 accuracy 0.864



Val   loss 0.3254726273007691 accuracy 0.87

Epoch 3/5
----------


...


Val   loss 0.4319960339926183 accuracy 0.8300000000000001

Epoch 4/5
----------



Train loss 0.0639086696319282 accuracy 0.982



Val   loss 0.5208611574489623 accuracy 0.8300000000000001

Epoch 5/5
----------



Train loss 0.01748604617023375 accuracy 0.996



Val   loss 0.4388579736405518 accuracy 0.9

